In [1]:
param_date = '20240623'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/1.groupname.xlsx',
                       header=0,
                       dtype=str).fillna('')

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'navnode_fetch',
                        'payload': f'''{{"jsn":{{"tab":"extras","groupname":"{input_.loc[a, 'GroupName']}","nodetype":"groupname"}}}}''',
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                json_ = json.loads(resp)

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_json = html.xpath('//input[contains(@id, "jsn[")]/@value')

                # = = = = = = = = = = = = = = =

                list_subgroupname = [json.loads(json_)['subgroupname'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Tab No': int(input_.loc[a, 'Tab No']),
                                        'Tab': input_.loc[a, 'Tab'],
                                        'GroupName No': int(input_.loc[a, 'GroupName No']),
                                        'GroupName': input_.loc[a, 'GroupName'],
                                        'SubGroupName No': [i+1 for i in range(len(list_subgroupname))],
                                        'SubGroupName': list_subgroupname})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Tab No': int(input_.loc[a, 'Tab No']),
                                     'Tab': input_.loc[a, 'Tab'],
                                     'GroupName No': int(input_.loc[a, 'GroupName No']),
                                     'GroupName': input_.loc[a, 'GroupName']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'GroupName No']}. {input_.loc[a, 'GroupName']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['Tab No', 'GroupName No', 'SubGroupName No'], ascending=[True, True, True], ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/2.subgroupname-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['Tab No', 'GroupName No'], ascending=[True, True], ignore_index=True)
    output_error.to_excel(f'./{param_date}/subgroupname_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：26

[状态：ok，尝试次数：  1] - 20. Literature
[剩余数量：6] - [当前时间：16:59:52]

[状态：ok，尝试次数：  1] - 15. Hardware
[剩余数量：5] - [当前时间：16:59:52]

[状态：ok，尝试次数：  1] - 11. Engine
[剩余数量：4] - [当前时间：16:59:52]

[状态：ok，尝试次数：  1] - 9. Electrical-Connector
[剩余数量：3] - [当前时间：16:59:52]

[状态：ok，尝试次数：  1] - 2. Belt Drive
[剩余数量：2] - [当前时间：16:59:52]

[状态：ok，尝试次数：  1] - 10. Electrical-Switch & Relay
[剩余数量：1] - [当前时间：16:59:53]

[状态：ok，尝试次数：  1] - 6. Drivetrain
[剩余数量：0] - [当前时间：16:59:53]

[状态：ok，尝试次数：  1] - 7. Electrical
[剩余数量：0] - [当前时间：16:59:53]

[状态：ok，尝试次数：  1] - 18. Ignition
[剩余数量：0] - [当前时间：16:59:53]

[状态：ok，尝试次数：  1] - 13. Fuel & Air
[剩余数量：0] - [当前时间：16:59:53]

[状态：ok，尝试次数：  1] - 16. Heat & Air Conditioning
[剩余数量：0] - [当前时间：16:59:53]

[状态：ok，尝试次数：  1] - 14. Garage Equipment
[剩余数量：0] - [当前时间：16:59:53]

[状态：ok，尝试次数：  1] - 24. Transmission-Manual
[剩余数量：0] - [当前时间：16:59:53]

[状态：ok，尝试次数：  1] - 22. Suspension
[剩余数量：0] - [当前时间：16:59:54]

[状态：ok，尝试次数：  1] - 8. Electrical-Bulb & Socket
[剩余数量：0] - [当前时间：16:59:54]

[状态：ok，尝